# 遗传算法
- 模拟自然界生物体的进化过程，主要分为3个阶段：选择、交叉、变异
- 首先使用一张原始图片，将其送入上述的退化模型中，得到模拟现实生活中的退化图片，作为目标。然后开始种群初始化，生成 n 个 [3，3] 的卷积核，作为种群。我们的目标就是通过将该种群的个体不断进化，得到一个优秀的卷积核，将原始图片与该卷积核卷积，得到的图片与目标图片非常相似。这里需要注意，我们生成的卷积核是 [3，3] 的，但由于图片是彩色图片，所以在卷积时卷积核也应该是三个通道的，暂定三个通道应该是相同的。为了保持卷积前后图片的亮度保持不变，卷积核应该进行归一化处理
- 适应度函数选择卷积后的图片与目标图片的峰值信噪比来作为个体的适应度
- 峰值信噪比表示信号的最大可能功率与影响信号表示精度的干扰噪声功率之间的比值

In [ ]:
import numpy as np
import random

In [ ]:
def creat_ck(nums, size=3):
    ck_list = []
    for num in range(nums):
        temp = np.zeros((size, size, 3))
        for i in range(size):
            for j in range(size):
                temp[i, j, :] = random.random()
        temp = 3 * temp / np.sum(temp)
        ck_list.append(temp)
    return ck_list

In [ ]:
def conv(img, ker):
    width, height, _ = img.shape
    timg = np.zeros((img.shape))
    for i in range(width):
        for j in range(height):
            if i + 3 > height:
                timg[i, j] = img[i, j]
                continue
            if j + 3 > height:
                timg[i, j] = img[i, j]
                continue

            temp = img[i : i + 3, j : j + 3]
            res = temp * ker

            timg[i, j, 0] = np.sum(res[:, :, 0])
            timg[i, j, 1] = np.sum(res[:, :, 1])
            timg[i, j, 2] = np.sum(res[:, :, 2])
    return np.uint8(timg)

In [ ]:
def psnr(img, targ, pixel_max=255):
    mses = np.mean((img - targ) ** 2)
    if mses == 0:
        return 100
    return 20 * np.log10(pixel_max / np.sqrt(mses))

## 选择
- 选择操作是把种群中适应度较大的个体挑出来，进化到下一代，这符合达尔文适者生存的观念。这里我们使用轮盘赌选择法，将各个个体的适应度算出来进行累加，作为该个体在轮盘赌中选择的概率。举个例子，假设我们有5个个体，每个个体的适应度为13，25，7，30，2。我们将其累加得到13，38，45，75，77，归一化后得到0.1688，0.4935，0.5844，0.9740，1。这时，第一个个体在轮盘上所占的空间为0到0.1688，第二个个体在轮盘上所占的空间为0.1688到0.4945...这样也就表明了适应度越大的个体在轮盘上所占的空间就越大，越容易被选中。我们产生一个0到1之间的随机数，假设为0.32，它介于0.1688到0.4945之间，所以我们本次选择第2个个体。轮盘赌选择的特点是，适应度越大的个体越容易被选择中，但并不是一定被选中。需要注意的是，在本问题中，我们计算峰值信噪比大概分布在24-28之间，相差并不是很大，为了将适应度拉开，我们选择计算峰值信噪比之后，放入指数函数中，这样细微的差距也能被分得很开

In [ ]:
def selection(zhong, img, tar):
    psnr_list = []
    for i in range(len(zhong)):
        res = conv(img, zhong[i])
        psnr_list.append(np.exp(psnr(res, tar)))
    psnr_list = np.array(psnr_list)
    psnr_list = psnr_list / np.sum(psnr_list)
    se_list = [0]
    sum_ = 0
    for i in range(len(zhong)):
        sum_ = sum_ + psnr_list[i]
        se_list.append(sum_)
    n_zhong = []
    for i in range(len(zhong)):
        t = random.random()
        for j in range(len(zhong)):
            if t >= se_list[j] and t < se_list[j + 1]:
                n_zhong.append(zhong[j])

    return n_zhong

## 交叉
- 高中生物里面应该讲过交叉操作，两条染色体交换部分片段的操作成为交叉。在遗传算法中，交叉操作为种群经过选择后，两两相交，产生新的后代。回到本问题中，交叉操作为经过选择操作的卷积核种群两两之间交换部分片段。由于卷积核是二维的（三个通道是相同的，所以我们简单将其视作二维），我们需要先将其展成一维，然后与一般的遗传算法相同，可以单点交叉，也可以片段交叉。单点交叉就是在个体串中随机选择一位，将该位之后的内容进行交叉。片段交叉就是在个体串中随机选择两位，在这两位之间的片段进行交叉。交叉完毕后，我们将个体卷积核恢复到二维。需要注意的是（之前也提高过这个问题），为了保持图片的亮度不变，交叉完毕后得到的新个体都需要重新进行归一化处理


In [ ]:
def cross(zhong):
    length = len(zhong)
    n_zhong = []
    for i in range(int(length / 2)):
        p1 = zhong[2 * i]
        p2 = zhong[2 * i + 1]
        t = p1.shape[0]
        p1 = p1.reshape((t * t, 3))
        p2 = p2.reshape((t * t, 3))
        loc = random.randint(0, t * t - 1)
        len_ = random.randint(loc, t * t - 1)

        temp = p1[loc:len_, :]
        p1[loc:len_, :] = p2[loc:len_, :]
        p2[loc:len_, :] = temp
        p1 = p1.reshape((t, t, 3))
        p2 = p2.reshape((t, t, 3))
        p1 = 3 * p1 / np.sum(p1)
        p2 = 3 * p2 / np.sum(p2)
        n_zhong.append(p1)
        n_zhong.append(p2)
    return n_zhong

## 变异
- 变异很好理解，就是改变染色体中的某一位基因。在现实生活中，变异的发生是非常小的，所以在算法中发生的概率也很小。我们首先需要设置一个变异的门槛，也就是变异发生的概率，这个值通常很小，大概在0.01到0.001之间。然后对每一个个体，都生成一个介于0到1的随机数，与我们设置的门槛比较，若其小于门槛，则表示该个体发生变异，这时我们随机改变该个体卷积核的其中一位；若其大于门槛，则表示该个体不发生变异，无事发生。还是老问题，为了保持图像的亮度不发生变化，在变异发生后我们需要对变异的卷积核进行归一化。当然，这样的操作会改变该个体卷积核中的所有数，目前我还没有想到什么更好的方法来解决该问题


In [ ]:
def change(zhong):
    chan_rate = 0.005
    width, height, _ = zhong[0].shape
    for i in range(len(zhong)):
        t = random.random()
        if t <= chan_rate:
            temp = zhong[i].reshape((width * height, 3))
            loc = random.randint(0, width * height - 1)
            temp[loc, :] = random.random()
            temp = 3 * temp / np.sum(temp)
            zhong[i] = temp.reshape((width, height, 3))
            # print(i, loc)
    return zhong